# ntp-atlas-pcap

* So Marco Davids found another bug with [Atlas NTP  measurements](https://mailman.ripe.net/archives/list/ripe-atlas@ripe.net/thread/33KNP7KPXN3YKUSEFTQGJMKVEHJAKX3Q/)
* IN short: most probes do not report results against his server
* He collected some pcap from a measurement
* I'll compare it here to see if they match


In [32]:
# First, we get atlas results from a measurement he ran:
#now let's get the ripe atlas measurements
import requests

#let's get measurements reported by Marco Davids on 

url = "https://atlas.ripe.net/api/v2/measurements/117140385/results/?format=json"

try:
    response = requests.get(url)
    response.raise_for_status()  # Raise exception for HTTP errors
    data_atlas = response.json()       # Parse JSON into a Python dict
    print(f"Downloaded {len(data_atlas)} records.")
except requests.RequestException as e:
    print(f"Error downloading data: {e}")

# 'data' now contains the JSON response as a Python dictionary/list


Downloaded 1 records.


In [30]:
# as can be seen, there is only one response in this
# however, the web site of the measurement has 50 probes
data_atlas

[{'fw': 5090,
  'mver': '2.6.4',
  'lts': 7,
  'dst_name': 'ntp0.testdns.nl',
  'ttr': 378.727243,
  'dst_addr': '141.105.125.85',
  'src_addr': '45.142.246.158',
  'proto': 'UDP',
  'af': 4,
  'li': 'no',
  'version': 4,
  'mode': 'server',
  'stratum': 1,
  'poll': 1024,
  'precision': 1.86265e-09,
  'root-delay': 0,
  'root-dispersion': 1.52588e-05,
  'ref-id': 'XFUN',
  'ref-ts': 3961208152.9128127,
  'result': [{'origin-ts': 3961208212.893019,
    'receive-ts': 3961208212.9098125,
    'transmit-ts': 3961208212.9128127,
    'final-ts': 3961208212.931933,
    'rtt': 0.035914,
    'offset': 0.001163},
   {'precision': 1.49012e-08,
    'ref-ts': 3961208152.951699,
    'origin-ts': 3961208212.931956,
    'receive-ts': 3961208212.9466987,
    'transmit-ts': 3961208212.951699,
    'final-ts': 3961208212.971092,
    'rtt': 0.034136,
    'offset': 0.002325},
   {'precision': 2.98023e-08,
    'ref-ts': 3961208152.9931803,
    'origin-ts': 3961208212.973435,
    'receive-ts': 3961208212.9911

In [33]:
# so which probes were used? 



# Replace with your actual measurement ID
measurement_id = 1234567

# API endpoint with optional fields
url = f"https://atlas.ripe.net/api/v2/measurements/117140385/?optional_fields=probes"

# Make the request
response = requests.get(url)
data = response.json()

# Extract probe IDs and store in a set
probe_ids = {probe["id"] for probe in data.get("probes", [])}

# Output the result
print(f"Number of unique probes: {len(probe_ids)}")
print(probe_ids)


Number of unique probes: 50
{12163, 1007620, 55817, 1006605, 1006606, 53390, 60049, 1008530, 660, 62589, 51350, 1003161, 1004827, 1006239, 1004704, 7072, 17441, 1010855, 50350, 13745, 54580, 29239, 1009593, 1010876, 1004862, 51007, 53953, 1003331, 22084, 1004746, 10443, 18891, 1007439, 62037, 1010904, 60123, 3804, 29793, 3811, 1009253, 55653, 10986, 61168, 1003762, 14194, 52980, 19832, 6905, 61820, 21373}


In [34]:
# let's now get metadata about these probes
import requests
import time

# Dict to store metadata
probe_metadata = {}

# Loop through probe IDs
for probe_id in probe_ids:
    url = f"https://atlas.ripe.net/api/v2/probes/{probe_id}/"
    response = requests.get(url)
    if response.status_code == 200:
        probe_metadata[probe_id] = response.json()
    else:
        print(f"Failed to fetch probe {probe_id}: HTTP {response.status_code}")
    time.sleep(0.1)  # Sleep to avoid hitting rate limits

# Print example output
for pid, meta in probe_metadata.items():
    print(f"{pid}: {meta.get('country_code')}, ASN: {meta.get('asn_v4')}")


12163: SG, ASN: 41095
1007620: NL, ASN: 12859
55817: SE, ASN: 8473
1006605: ME, ASN: 8585
1006606: BE, ASN: 57225
53390: US, ASN: 7922
60049: BR, ASN: 52863
1008530: GB, ASN: 2856
660: ES, ASN: 57269
62589: AR, ASN: 3549
51350: US, ASN: 22773
1003161: US, ASN: 10796
1004827: US, ASN: 46375
1006239: CL, ASN: 270024
1004704: US, ASN: 53698
7072: RU, ASN: 57578
17441: SE, ASN: 50821
1010855: US, ASN: 22442
50350: NL, ASN: 41153
13745: BE, ASN: 6848
54580: RU, ASN: 29124
29239: FR, ASN: 12322
1009593: NL, ASN: 50266
1010876: US, ASN: 20473
1004862: US, ASN: 26370
51007: GR, ASN: 3329
53953: US, ASN: 32890
1003331: DE, ASN: 31898
22084: NL, ASN: 50266
1004746: RS, ASN: 25431
10443: US, ASN: 13977
18891: NZ, ASN: 56030
1007439: IT, ASN: 202613
62037: US, ASN: 701
1010904: BG, ASN: 26383
60123: AT, ASN: 1853
3804: RU, ASN: 28738
29793: DK, ASN: 216275
3811: RU, ASN: 42610
1009253: CN, ASN: 9808
55653: BE, ASN: 6848
10986: NL, ASN: 1103
61168: TW, ASN: 3462
1003762: NL, ASN: 50266
14194: DE, A

In [35]:
#now let's make an probe_id iP-address dict
ip_probe_id=dict()

for p, values in probe_metadata.items():
    ip_probe_id[values['address_v4']]=p

In [110]:
ip_probe_id

# Invert the dictionary
probe_id_ip = {v: k for k, v in ip_probe_id.items()}

print(probe_id_ip)

{12163: '103.242.74.213', 1007620: '213.136.26.43', 55817: '85.24.232.250', 1006605: '95.155.57.115', 1006606: '193.150.106.30', 53390: '67.169.21.122', 10986: None, 1008530: '86.128.46.71', 62589: '190.216.95.53', 51350: '68.104.118.208', 1003161: '74.140.111.161', 1004827: '23.93.27.73', 1006239: '200.39.138.190', 1004704: '23.152.192.43', 7072: '45.142.246.158', 1010855: '66.60.237.66', 50350: '91.215.7.190', 13745: '94.224.11.87', 54580: '81.200.31.214', 29239: '82.66.187.120', 1009593: '178.226.252.165', 1010876: '137.220.60.24', 1004862: '130.51.193.230', 53953: '206.174.163.52', 1003331: '130.162.35.116', 22084: '188.90.208.189', 1004746: '217.24.20.4', 10443: '70.20.54.168', 18891: '114.23.160.229', 1007439: '217.198.140.2', 62037: '100.4.110.245', 1010904: '31.13.248.138', 60123: '78.104.195.8', 3804: '78.153.134.4', 29793: '91.226.145.64', 3811: '46.242.12.248', 1009253: '112.44.189.239', 55653: '94.224.191.57', 61168: '61.228.65.92', 1003762: '85.146.98.97', 14194: '212.111.

In [36]:
from scapy.all import rdpcap, UDP, IP
import struct

In [24]:
#now let' s look at the pcap collected by Marco on his server side
# this was done during the measurements ran by atlas
# it may have more clients than atlas only


In [37]:


PCAP_FILE = 'ntp0.pcap'
NTP_PORT = 123

def get_ntp_timestamp(data: bytes, offset: int) -> float:
    """Extract 64-bit NTP timestamp starting at `offset` in payload."""
    if len(data) < offset + 8:
        return 0.0
    seconds, fraction = struct.unpack("!II", data[offset:offset+8])
    return seconds + fraction / 2**32

def parse_ntp_packets(pcap_path):
    packets = rdpcap(pcap_path)

    requests = {}
    responses = []
    matched = []

    for pkt in packets:
        if pkt.haslayer(UDP) and pkt.haslayer(IP):
            udp = pkt[UDP]
            ip = pkt[IP]

            if udp.sport == NTP_PORT or udp.dport == NTP_PORT:
                payload = bytes(udp.payload)

                if len(payload) < 48:
                    continue  # malformed NTP packet

                src = ip.src
                dst = ip.dst

                if udp.dport == NTP_PORT:
                    # This is an NTP request
                    tx_ts = get_ntp_timestamp(payload, 40)  # Transmit timestamp
                    key = (src, dst, tx_ts)
                    requests[key] = {
                        "src": src,
                        "dst": dst,
                        "tx_ts": tx_ts,
                        "raw": payload,
                        "time": pkt.time
                    }
                elif udp.sport == NTP_PORT:
                    # This is an NTP response
                    org_ts = get_ntp_timestamp(payload, 24)  # Originate timestamp
                    key = (dst, src, org_ts)  # reversed IPs
                    response = {
                        "src": src,
                        "dst": dst,
                        "org_ts": org_ts,
                        "raw": payload,
                        "time": pkt.time
                    }
                    if key in requests:
                        matched.append({
                            "request": requests[key],
                            "response": response
                        })
                        del requests[key]
                    else:
                        responses.append(response)

    return matched

# do the match queries_repsonses
ntp_matches = parse_ntp_packets(PCAP_FILE)


In [26]:
ntp_matches

[{'request': {'src': '45.142.246.158',
   'dst': '141.105.125.85',
   'tx_ts': 3961208212.893019,
   'raw': b'#\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xec\x1b=\x94\xe4\x9c\xe4\xa7',
   'time': Decimal('1752219412.912646344')},
  'response': {'src': '141.105.125.85',
   'dst': '45.142.246.158',
   'org_ts': 3961208212.893019,
   'raw': b'$\x01\n\xe3\x00\x00\x00\x00\x00\x00\x00\x01XFUN\xec\x1b=X\xe9\xae\x16(\xec\x1b=\x94\xe4\x9c\xe4\xa7\xec\x1b=\x94\xe8\xe9z\x82\xec\x1b=\x94\xe9\xae\x16(',
   'time': Decimal('1752219412.912833286')}},
 {'request': {'src': '45.142.246.158',
   'dst': '141.105.125.85',
   'tx_ts': 3961208212.931956,
   'raw': b'#\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xec\x1b=\x94\xee\x94\xab\x1d',
   'time': Decimal('1752219412.951492593')},


In [38]:
# let' see if we can find the query from the probe that returned results 


[{'fw': 5090,
  'mver': '2.6.4',
  'lts': 7,
  'dst_name': 'ntp0.testdns.nl',
  'ttr': 378.727243,
  'dst_addr': '141.105.125.85',
  'src_addr': '45.142.246.158',
  'proto': 'UDP',
  'af': 4,
  'li': 'no',
  'version': 4,
  'mode': 'server',
  'stratum': 1,
  'poll': 1024,
  'precision': 1.86265e-09,
  'root-delay': 0,
  'root-dispersion': 1.52588e-05,
  'ref-id': 'XFUN',
  'ref-ts': 3961208152.9128127,
  'result': [{'origin-ts': 3961208212.893019,
    'receive-ts': 3961208212.9098125,
    'transmit-ts': 3961208212.9128127,
    'final-ts': 3961208212.931933,
    'rtt': 0.035914,
    'offset': 0.001163},
   {'precision': 1.49012e-08,
    'ref-ts': 3961208152.951699,
    'origin-ts': 3961208212.931956,
    'receive-ts': 3961208212.9466987,
    'transmit-ts': 3961208212.951699,
    'final-ts': 3961208212.971092,
    'rtt': 0.034136,
    'offset': 0.002325},
   {'precision': 2.98023e-08,
    'ref-ts': 3961208152.9931803,
    'origin-ts': 3961208212.973435,
    'receive-ts': 3961208212.9911

In [41]:
reporting_probes=dict()
for k in data_atlas:
    #let's get the probe_ip_address
    probe_ip=( k['src_addr'])
    tempL=[]
    for m in ntp_matches:
        if probe_ip in m['request']['src']:
            tempL.append(m['request'])
    reporting_probes[ip_probe_id[probe_ip]]=tempL
            

In [89]:
reporting_probes

{7072: [{'src': '45.142.246.158',
   'dst': '141.105.125.85',
   'tx_ts': 3961208212.893019,
   'raw': b'#\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xec\x1b=\x94\xe4\x9c\xe4\xa7',
   'time': Decimal('1752219412.912646344')},
  {'src': '45.142.246.158',
   'dst': '141.105.125.85',
   'tx_ts': 3961208212.931956,
   'raw': b'#\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xec\x1b=\x94\xee\x94\xab\x1d',
   'time': Decimal('1752219412.951492593')},
  {'src': '45.142.246.158',
   'dst': '141.105.125.85',
   'tx_ts': 3961208212.973435,
   'raw': b'#\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xec\x1b=\x94\xf93\tA',
   'time': Decimal('1752219412.992997945')}]}

In [50]:
data_atlas

[{'fw': 5090,
  'mver': '2.6.4',
  'lts': 7,
  'dst_name': 'ntp0.testdns.nl',
  'ttr': 378.727243,
  'dst_addr': '141.105.125.85',
  'src_addr': '45.142.246.158',
  'proto': 'UDP',
  'af': 4,
  'li': 'no',
  'version': 4,
  'mode': 'server',
  'stratum': 1,
  'poll': 1024,
  'precision': 1.86265e-09,
  'root-delay': 0,
  'root-dispersion': 1.52588e-05,
  'ref-id': 'XFUN',
  'ref-ts': 3961208152.9128127,
  'result': [{'origin-ts': 3961208212.893019,
    'receive-ts': 3961208212.9098125,
    'transmit-ts': 3961208212.9128127,
    'final-ts': 3961208212.931933,
    'rtt': 0.035914,
    'offset': 0.001163},
   {'precision': 1.49012e-08,
    'ref-ts': 3961208152.951699,
    'origin-ts': 3961208212.931956,
    'receive-ts': 3961208212.9466987,
    'transmit-ts': 3961208212.951699,
    'final-ts': 3961208212.971092,
    'rtt': 0.034136,
    'offset': 0.002325},
   {'precision': 2.98023e-08,
    'ref-ts': 3961208152.9931803,
    'origin-ts': 3961208212.973435,
    'receive-ts': 3961208212.9911

In [42]:
reporting_probes

{7072: [{'src': '45.142.246.158',
   'dst': '141.105.125.85',
   'tx_ts': 3961208212.893019,
   'raw': b'#\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xec\x1b=\x94\xe4\x9c\xe4\xa7',
   'time': Decimal('1752219412.912646344')},
  {'src': '45.142.246.158',
   'dst': '141.105.125.85',
   'tx_ts': 3961208212.931956,
   'raw': b'#\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xec\x1b=\x94\xee\x94\xab\x1d',
   'time': Decimal('1752219412.951492593')},
  {'src': '45.142.246.158',
   'dst': '141.105.125.85',
   'tx_ts': 3961208212.973435,
   'raw': b'#\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xec\x1b=\x94\xf93\tA',
   'time': Decimal('1752219412.992997945')}]}

In [65]:
# we see that there are 3 requests fromt his probe 
# on the pcap file, and they are also seen on data_atlas['result']
#see if they really match

#fisrst get the tx_ts fromthe pcaps and see if they are there
tx_times=[]
for probz,values in reporting_probes.items():
 
    #print(probz)
    for m_probes in values:

        tx_times.append(m_probes['tx_ts'])
    #now let' see if they match the data_atlas
tx_times

[3961208212.893019, 3961208212.931956, 3961208212.973435]

In [90]:
ntp_matches

[{'request': {'src': '45.142.246.158',
   'dst': '141.105.125.85',
   'tx_ts': 3961208212.893019,
   'raw': b'#\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xec\x1b=\x94\xe4\x9c\xe4\xa7',
   'time': Decimal('1752219412.912646344')},
  'response': {'src': '141.105.125.85',
   'dst': '45.142.246.158',
   'org_ts': 3961208212.893019,
   'raw': b'$\x01\n\xe3\x00\x00\x00\x00\x00\x00\x00\x01XFUN\xec\x1b=X\xe9\xae\x16(\xec\x1b=\x94\xe4\x9c\xe4\xa7\xec\x1b=\x94\xe8\xe9z\x82\xec\x1b=\x94\xe9\xae\x16(',
   'time': Decimal('1752219412.912833286')}},
 {'request': {'src': '45.142.246.158',
   'dst': '141.105.125.85',
   'tx_ts': 3961208212.931956,
   'raw': b'#\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xec\x1b=\x94\xee\x94\xab\x1d',
   'time': Decimal('1752219412.951492593')},


In [101]:
# double chekcing, to see if it they really match also in ip address and timestamp
# the origin-ts at atlas is the tx_time in the pcap


for k in data_atlas:
        #let's get the probe_ip_address
        probe_ip=( k['src_addr'])
        tempL=[]
        # only if the ip matches
        for m in ntp_matches:
            
            #match atlas probe ip with rqeuests seen at the ntp server
            if probe_ip in m['request']['src']:
                
                print(m['request'])
                for s in k['result']:
                   
                    if s['origin-ts'] ==   m['request']['tx_ts']:
                        print('match')
                
               

{'src': '45.142.246.158', 'dst': '141.105.125.85', 'tx_ts': 3961208212.893019, 'raw': b'#\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xec\x1b=\x94\xe4\x9c\xe4\xa7', 'time': Decimal('1752219412.912646344')}
match
{'src': '45.142.246.158', 'dst': '141.105.125.85', 'tx_ts': 3961208212.931956, 'raw': b'#\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xec\x1b=\x94\xee\x94\xab\x1d', 'time': Decimal('1752219412.951492593')}
match
{'src': '45.142.246.158', 'dst': '141.105.125.85', 'tx_ts': 3961208212.973435, 'raw': b'#\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xec\x1b=\x94\xf93\tA', 'time': Decimal('1752219412.992997945')}
match


In [117]:
#now get all the queries from ip addresses from atlas probes seen on the pcap but not on atlas
missing=[]
missing_probe_id=dict()

for m in ntp_matches:
    if m['request']['src'] in ip_probe_id.keys()  and m['request']['src'] !='45.142.246.158':
        missing.append(m)
        temp_probe_id=ip_probe_id[ m['request']['src']]
        tempL=[]
        if temp_probe_id in missing_probe_id:
            tempL=missing_probe_id[temp_probe_id]
        tempL.append(m)
        missing_probe_id[temp_probe_id]=tempL
        

In [124]:
import json
from decimal import Decimal

def clean_for_json(obj):
    """Recursively convert bytes to hex strings and Decimals to floats."""
    if isinstance(obj, dict):
        return {k: clean_for_json(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [clean_for_json(i) for i in obj]
    elif isinstance(obj, bytes):
        return obj.hex()
    elif isinstance(obj, Decimal):
        return float(obj)
    else:
        return obj
    
    

In [125]:

# Clean and save to JSON
cleaned_data = clean_for_json(missing_probe_id)

with open("missing_probe_id.json", "w") as f:
    json.dump(cleaned_data, f, indent=2)


In [126]:
!cat missing_probe_id.json

{
  "6905": [
    {
      "request": {
        "src": "103.253.132.25",
        "dst": "141.105.125.85",
        "tx_ts": 3961208214.737752,
        "raw": "23000000000000000000000000000000000000000000000000000000000000000000000000000000ec1b3d96bcdd50a8",
        "time": 1752219414.8317049
      },
      "response": {
        "src": "141.105.125.85",
        "dst": "103.253.132.25",
        "org_ts": 3961208214.737752,
        "raw": "240108e800000000000000015846554eec1b3d5ad4f7fc9dec1b3d96bcdd50a8ec1b3d96d474ea2fec1b3d96d4f7fc9d",
        "time": 1752219414.831933
      }
    },
    {
      "request": {
        "src": "103.253.132.25",
        "dst": "141.105.125.85",
        "tx_ts": 3961208214.944847,
        "raw": "23000000000000000000000000000000000000000000000000000000000000000000000000000000ec1b3d96f1e17e34",
        "time": 1752219415.0388045
      },
      "response": {
        "src": "141.105.125.85",
        "dst": "103.253.132.25",
        "org_ts": 3961208214.944847,
    

In [127]:
len(cleaned_data)

44